# Satellite Image Pre-processing for Land Use Change Detection

## Overview
This notebook handles the preprocessing of satellite imagery for detecting land use changes in Córdoba, Argentina. It uses Google Earth Engine (GEE) to access and process satellite data.

## Features
- Connects to Google Earth Engine API
- Defines study area in Córdoba region
- Retrieves Sentinel-2 imagery
- Applies cloud masking and filtering
- Generates composite images for analysis

## Requirements
- Google Earth Engine account
- Python packages:
  - `ee` (Earth Engine API)
  - `geemap` (for interactive mapping)
  - `folium` (for map visualization)

## Usage
1. Run the authentication cell to connect to GEE
2. Define your study area parameters
3. Set date ranges for image collection
4. Execute preprocessing steps
5. Export processed images for analysis

## Output
- Processed satellite images
- Interactive maps of the study area
- Preprocessed data ready for change detection analysis

## Author
Mujtaba

## Project
Part of the Córdoba Argentina Chapter - Monitoring Land Use Transformation project

In [2]:
import ee
import geemap
import folium
from folium import plugins


In [4]:
ee.Authenticate(auth_mode='colab')

True

In [5]:
ee.Initialize(project='ee-mujtabanaqvi29')

In [9]:
import ee
import geemap
from datetime import datetime
# Authenticate and initialize GEE
ee.Authenticate()
ee.Initialize()

def detect_deforestation_with_report(region, start_date_before, end_date_after, ndvi_threshold=-0.15):
    """
    Detects deforestation and generates a detailed report including:
    - Dates of analysis
    - Duration
    - Total investigated area
    - Total deforested area (in hectares)
    """
    # Load Sentinel-2 imagery
    before = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(region) \
        .filterDate(start_date_before, end_date_after) \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median()

    after = ee.ImageCollection('COPERNICUS/S2') \
        .filterBounds(region) \
        .filterDate(end_date_after, '2023-12-31') \
        .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
        .median()

    # Calculate NDVI for before and after
    ndvi_before = before.normalizedDifference(['B8', 'B4']).rename('NDVI_Before')
    ndvi_after = after.normalizedDifference(['B8', 'B4']).rename('NDVI_After')

    # Calculate NDVI change
    ndvi_change = ndvi_after.subtract(ndvi_before).rename('NDVI_Change')

    # Apply threshold to detect deforestation
    deforestation = ndvi_change.lt(ndvi_threshold).selfMask()

    # Calculate the total deforested area
    deforestation_area = deforestation.multiply(ee.Image.pixelArea())
    total_deforestation_area = deforestation_area.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=10,
        maxPixels=1e9
    ).getInfo()

    # Convert to hectares
    deforested_area_ha = total_deforestation_area.get('NDVI_Change', 0) / 10_000

    # Calculate the total area of the region (in hectares)
    total_region_area = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=region,
        scale=10,
        maxPixels=1e9
    ).getInfo()
    total_region_area_ha = total_region_area.get('area') / 10_000

    # Calculate the duration of analysis
    start_date = datetime.strptime(start_date_before, '%Y-%m-%d')
    end_date = datetime.strptime(end_date_after, '%Y-%m-%d')
    duration_days = (end_date - start_date).days

    # Visualization parameters
    ndvi_viz = {'min': -1, 'max': 1, 'palette': ['blue', 'white', 'green']}
    change_viz = {'min': -0.3, 'max': 0.3, 'palette': ['red', 'yellow', 'green']}
    deforestation_viz = {'min': 0, 'max': 1, 'palette': ['red']}
    true_color_viz = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

    # Visualization
    Map = geemap.Map()
    Map.centerObject(region, 12)
    Map.addLayer(before, true_color_viz, 'Satellite Image Before')
    Map.addLayer(after, true_color_viz, 'Satellite Image After')
    Map.addLayer(ndvi_before, ndvi_viz, 'NDVI Before')
    Map.addLayer(ndvi_after, ndvi_viz, 'NDVI After')
    Map.addLayer(ndvi_change, change_viz, 'NDVI Change')
    Map.addLayer(deforestation, deforestation_viz, 'Deforestation Detected')

    # Generate the detailed report
    report = f"""
    Deforestation Analysis Report
    ------------------------------
    Region Investigated: Custom Region (User Defined)
    Total Area Investigated: {total_region_area_ha:.2f} hectares
    Analysis Dates:
        - Before: {start_date_before}
        - After: {end_date_after}
    Analysis Duration: {duration_days} days
    NDVI Threshold for Deforestation: {ndvi_threshold}
    Total Deforested Area: {deforested_area_ha:.2f} hectares
    """
    print(report)
    return Map

# Define the region (example: Córdoba region)
region = ee.Geometry.Polygon([
    [[-64.18, -31.40], [-64.17, -31.40], [-64.17, -31.41], [-64.18, -31.41], [-64.18, -31.40]]
])

# Detect deforestation and generate the report
Map = detect_deforestation_with_report(region, '2022-01-01', '2022-12-31', ndvi_threshold=-0.1)
Map



    Deforestation Analysis Report
    ------------------------------
    Region Investigated: Custom Region (User Defined)
    Total Area Investigated: 105.37 hectares
    Analysis Dates:
        - Before: 2022-01-01
        - After: 2022-12-31
    Analysis Duration: 364 days
    NDVI Threshold for Deforestation: -0.1
    Total Deforested Area: 0.12 hectares
    


Map(center=[-31.405000008271625, -64.1750000000164], controls=(WidgetControl(options=['position', 'transparent…